In [1]:
# %load ../../loaders/imports.py
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import time
import pdb

# Add the uoicorr directory to the path
sys.path.append('../../../uoicorr')

# Add the root directory of this repository
sys.path.append('../..')

from utils import gen_covariance, gen_beta2, gen_data, get_cov_list
from utils import selection_accuracy
from sklearn.linear_model import LassoLars, lasso_path, LinearRegression

from pyuoi.linear_model import UoI_Lasso

In [17]:
sigma = gen_covariance(100, 0.25, 10, 5, 1)

In [19]:
sigma

array([[1.00000000e+00, 8.18730753e-01, 6.70320046e-01, ...,
        3.75566677e-09, 3.07487988e-09, 2.51749872e-09],
       [8.18730753e-01, 1.00000000e+00, 8.18730753e-01, ...,
        4.58718175e-09, 3.75566677e-09, 3.07487988e-09],
       [6.70320046e-01, 8.18730753e-01, 1.00000000e+00, ...,
        5.60279644e-09, 4.58718175e-09, 3.75566677e-09],
       ...,
       [3.75566677e-09, 4.58718175e-09, 5.60279644e-09, ...,
        1.00000000e+00, 8.18730753e-01, 6.70320046e-01],
       [3.07487988e-09, 3.75566677e-09, 4.58718175e-09, ...,
        8.18730753e-01, 1.00000000e+00, 8.18730753e-01],
       [2.51749872e-09, 3.07487988e-09, 3.75566677e-09, ...,
        6.70320046e-01, 8.18730753e-01, 1.00000000e+00]])

In [23]:
lam = np.linalg.eig(sigma)[0]

In [26]:
G = np.linalg.eig(sigma)[1] @ np.diag(np.sqrt(lam - np.min(lam)))

array([[1.00000000e+00, 8.18730753e-01, 6.70320046e-01, ...,
        3.75566733e-09, 3.07488063e-09, 2.51749993e-09],
       [8.18730753e-01, 1.00000000e+00, 8.18730753e-01, ...,
        4.58718158e-09, 3.75566628e-09, 3.07488013e-09],
       [6.70320046e-01, 8.18730753e-01, 1.00000000e+00, ...,
        5.60279445e-09, 4.58717969e-09, 3.75566528e-09],
       ...,
       [3.75566733e-09, 4.58718158e-09, 5.60279445e-09, ...,
        1.00000000e+00, 8.18730753e-01, 6.70320046e-01],
       [3.07488063e-09, 3.75566628e-09, 4.58717969e-09, ...,
        8.18730753e-01, 1.00000000e+00, 8.18730753e-01],
       [2.51749993e-09, 3.07488013e-09, 3.75566528e-09, ...,
        6.70320046e-01, 8.18730753e-01, 1.00000000e+00]])

In [28]:
np.allclose(G @ G.T + np.min(lam) * np.eye(100), sigma)

True

In [9]:
sigma = 0.25 * np.ones((10, 10))

In [12]:
np.linalg.eig(sigma)[0]

array([0. , 2.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ])

In [13]:
np.linalg.eig(sigma)[1]

array([[-9.48683298e-01,  3.16227766e-01, -5.55111512e-18,
        -5.55111512e-18, -5.55111512e-18, -5.55111512e-18,
        -5.55111512e-18, -5.55111512e-18, -5.55111512e-18,
        -5.55111512e-18],
       [ 1.05409255e-01,  3.16227766e-01, -3.33333333e-01,
        -3.33333333e-01, -3.33333333e-01, -3.33333333e-01,
        -3.33333333e-01, -3.33333333e-01, -3.33333333e-01,
        -3.33333333e-01],
       [ 1.05409255e-01,  3.16227766e-01,  9.16666667e-01,
        -8.33333333e-02, -8.33333333e-02, -8.33333333e-02,
        -8.33333333e-02, -8.33333333e-02, -8.33333333e-02,
        -8.33333333e-02],
       [ 1.05409255e-01,  3.16227766e-01, -8.33333333e-02,
         9.16666667e-01, -8.33333333e-02, -8.33333333e-02,
        -8.33333333e-02, -8.33333333e-02, -8.33333333e-02,
        -8.33333333e-02],
       [ 1.05409255e-01,  3.16227766e-01, -8.33333333e-02,
        -8.33333333e-02,  9.16666667e-01, -8.33333333e-02,
        -8.33333333e-02, -8.33333333e-02, -8.33333333e-02,
        -8.

In [14]:
np.rank(sigma)

/home/akumar/anaconda3/envs/nse/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  """Entry point for launching an IPython kernel.


2

In [16]:
sigma

array([[0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25]])

In [21]:
sigma.T @ sigma

array([[0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25]])